# Local Quick Start

In [Cloud Quick Start](https://www.run.house/docs/tutorials/quick-start-cloud), we demonstrate how to deploy a local function to a remote cluster using Runhouse. In this local-only version, we show how to use Runhouse to set up a local web server, and deploy an arbitrary Python function to it.

## Runhouse Server Setup

In [ ]:
!pip install runhouse

First install Runhouse with `pip install runhouse`

Next, start the Runhouse server locally on CLI with `runhouse restart`, and use `runhouse status` to print the status and details of the server.

In [ ]:
!runhouse restart

In [3]:
!runhouse status

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/sashabelousovrh/Library/Application Support/sagemaker/config.yaml
😈 Runhouse Daemon is running 🏃
Runhouse v0.0.28
server pid: 8371
• server port: 32300
• den auth: False
• server connection type: none
• backend config:
  • resource subtype: Cluster
  • use local telemetry: False
  • domain: None
  • server host: 0.0.0.0
  • ips: ['0.0.0.0']
  • resource subtype: Cluster
Serving 🍦 :
• _cluster_default_env (runhouse.Env)
  This environment has only python packages installed, if such provided. No 
resources were found.


## Local Python Function

Let's first define a simple Python function that we want to send to the server. This function returns the process ID it runs on, and optionally takes in a parameter, which it adds to the process ID prior to returning it.

In [5]:
def get_pid(a=0):
    import os
    return os.getpid() + int(a)

## Deployment

Standing up your Python code on the server is simple with the Runhouse API. Wrap the function with `rh.function`, and then use `.to(rh.here)` to sync it to the server.

In [ ]:
import runhouse as rh

In [9]:
server_fn = rh.function(get_pid).to(rh.here)

INFO | 2024-02-26 22:14:53.460361 | Writing out function to /Users/caroline/Documents/runhouse/notebooks/docs/get_pid_fn.py. Please make sure the function does not rely on any local variables, including imports (which should be moved inside the function body).
INFO | 2024-02-26 22:14:53.523591 | Sending module get_pid to local Runhouse daemon


The `get_pid` function we defined above now exists on the server.

### Remote Function Call

You can call the server function just as you would any other Python function, with `server_fn()`, and it runs on the server and returns the result to our local environment.

Below, we run both the local and server versions of this function, which give different results and confirms that the functions are indeed being run on different processes.

In [11]:
print(f"Local PID {get_pid()}")
print(f"Server PID {server_fn()}")

Local PID 27818
Server PID 19846


### HTTP Endpoint and Curl

In addition to calling the function directly in Python, we can also access it with a curl call or open it up in a browser.

In [12]:
server_fn.endpoint()

'http://0.0.0.0:32300/get_pid'

In [13]:
!curl "http://0.0.0.0:32300/get_pid/call"

{"data":"19846","error":null,"traceback":null,"output_type":"result_serialized","serialization":"json"}

To pass in the optional function parameter:

In [14]:
!curl "http://0.0.0.0:32300/get_pid/call?a=1"

{"data":"19847","error":null,"traceback":null,"output_type":"result_serialized","serialization":"json"}